# Correlations Plot

In [ ]:
"""
    TITLE   : Correlations Plot
    PROJECT : discO
"""

__author__ = "Nathaniel Starkman"
__version__ = ""

<br><br>

- - - 

## Prepare


### Imports

In [2]:
# THIRD PARTY
import astropy.coordinates as coord
import astropy.units as u
from astropy.visualization import quantity_support
from galpy import df as gdf
from galpy import potential as gpot
import numpy as np
from IPython.display import display
import scipy.stats
import matplotlib.pyplot as plt
from IPython.display import display
import corner

# PROJECT-SPECIFIC
from discO import (
    PotentialWrapper,
    PotentialSampler,
    MeasurementErrorSampler,
    PotentialFitter,
    ResidualMethod,
    Pipeline
)
from discO.utils import UnFrame

In [3]:
quantity_support();

### Parameters

In [4]:
mass = 1e12 * u.solMass
r0 = 16 * u.kpc  # scale factor

In [5]:
# pot = gpot.MWPotential2014[-1]
# pot = gpot.NFWPotential(amp=mass, a=r0)
pot = gpot.HernquistPotential(amp=2*mass, a=r0)
pot.turn_physical_on()

In [6]:
frame = coord.Galactocentric()

<br><br>

- - - 

## Potential Wrapper

In [7]:
wpot = PotentialWrapper(pot, frame="galactocentric")
wpot

GalpyPotentialWrapper: at <0x7f9988ae8940>
    potential : galpy.potential.TwoPowerSphericalPotential.HernquistPotential object at 0x7f9988ae8040
    frame     : Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
        (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)

<br><br>

- - - 

## Potential Sampler

In [8]:
sampler = PotentialSampler(
    wpot,
    df=gdf.isotropicHernquistdf
)
sampler.frame

<Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>

In [9]:
samples = sampler.run(n=int(1e3), iterations=200, batch=True)
samples.shape

100%|██████████| 200/200 [00:00<00:00, 358.79it/s]


(1000, 200)

<br><br>

- - - 

## Measurement

In [10]:
measurer = MeasurementErrorSampler(
    c_err=10 * u.percent, method="Gaussian",
    frame="icrs",
    representation_type="spherical"
)
measurer

In [11]:
for resamp in measurer.run(samples, progress=True):
    pass

100%|██████████| 200/200 [00:01<00:00, 126.49it/s]


<br><br>

- - - 

## Potential Fitter

In [12]:
fitter = PotentialFitter(
    key="galpy", potential_cls="scf",
    frame=sampler.frame,
    Nmax=3, Lmax=3, scale_factor=r0
)
fitter

	frame: <Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>
	defaults: {'Nmax': 3, 'Lmax': 3, 'scale_factor': <Quantity 16. kpc>}

In [13]:
fits = fitter.run(samples, Nmax=10, Lmax=10, scale_factor=r0, batch=True)

100%|██████████| 200/200 [00:02<00:00, 72.61it/s]


In [34]:
for fit in fitter.run(samples, Nmax=10, Lmax=10, scale_factor=r0):
    pass

100%|██████████| 200/200 [00:03<00:00, 62.73it/s]


In [14]:
fits[0]

GalpyPotentialWrapper: at <0x7f9988ae83d0>
    potential : galpy.potential.SCFPotential.SCFPotential object at 0x7f9988ae8160
    frame     : Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
        (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)

<br><br>

- - - 

## Residual

In [15]:
_r = np.linspace(0.1, 10, num=50)
_lon = np.linspace(0, 360, num=10)
_lat = np.linspace(-90, 90, num=10)
r, lon, lat = np.meshgrid(_r, _lon, _lat)
grid = coord.Galactocentric(
    coord.SphericalRepresentation(lon * u.deg, lat * u.deg, r * u.kpc)
)

In [16]:
residualer = ResidualMethod(
    grid,
    method="grid",
    original_potential=wpot,
    observable="acceleration",
    representation_type="cartesian"
)

In [30]:
residualer(fits[0])

/Users/nathanielstarkman/miniconda3/envs/dev/lib/python3.8/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



<CartesianVectorField (x, y, z) in kpc | (vf_x, vf_y, vf_z) in km / s2
    [((6.12323400e-18, 0., -0.1       ), (           nan,             nan,             nan)),
     ((3.42020143e-02, 0., -0.09396926), (3.44900643e-13, -1.13348518e-13,  5.06768420e-13)),
     ((6.42787610e-02, 0., -0.07660444), (3.48034513e-13, -1.46441766e-13,  5.21838085e-13)),
     ...,
     ((6.42787610e+00, 0.,  7.66044443), (5.36561402e-14, -1.71906784e-14, -3.42988336e-14)),
     ((3.42020143e+00, 0.,  9.39692621), (4.44588587e-14, -1.07417254e-14,  3.70116599e-15)),
     ((6.12323400e-16, 0., 10.        ), (           nan,             nan,             nan))]>

<br><br>

- - - 

## Pipeline

In [19]:
pipe = Pipeline(
    sampler=sampler,
    measurer=measurer,
    fitter=fitter,
)
pipe

Pipeline:
    sampler: <discO.plugin.galpy.sample.GalpyPotentialSampler object at 0x7f9984f7a130>
    measurer: <discO.core.measurement.GaussianMeasurementError object at 0x7f9988b28f40>
    fitter: <discO.plugin.galpy.fitter.GalpySCFPotentialFitter object at 0x7f9988b31220>
	frame: <Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>
	defaults: {'Nmax': 3, 'Lmax': 3, 'scale_factor': <Quantity 16. kpc>}
    residual: None
    statistic: None

In [20]:
coeff_array1 = []
coeff_array2 = []

numphis = 27  # (N=3 x (L=3)^2)
idx = [
    i for i in np.arange(numphis)
    if i not in [1, 2, 5, 10, 11, 14, 19, 20, 23]
]

plot_options=dict(
    labels=[r"$\phi_{" + str(i) + r"}$" for i in idx],
    show_titles=True,
    quantiles=[0.16, 0.5, 0.84],
    title_kwargs={"fontsize": 12},
    title_fmt='.2e',
    label_kwargs={"fontsize":12},
    hist2d_kwargs={"fontsize":8},
    quiet=True
)

# Run for 10k samples, for 10k iterations
N = int(1e4)
iterations = int(1e4)
random=0
stride = 5000

for i, result1 in enumerate(pipe.run(N, iterations, random=random, c_err=False)):
    # the no-observer coefficients
    coeff_array1.append(result1.fit.coefficients()["Acos"].flatten())

    result2 = pipe(result1.sample, random=random)
    coeff_array2.append(result2.fit.coefficients()["Acos"].flatten())

    if (i + 1) % stride == 0 and (i + 1) > numphis:

        # horribly inefficient saving
        np.save("figures/atcenter", coeff_array1)
        np.save("figures/atobserver", coeff_array2)

        # plot
        plt.close("all")
        fig = corner.corner(np.array(coeff_array1)[:, idx], **plot_options)
        fig.suptitle("At Center", y=1.00)
        fig.savefig(f"figures/plot_{i+1}_atcenter.png")
        plt.close(fig)

        fig = corner.corner(np.array(coeff_array2)[:, idx], **plot_options)
        fig.suptitle("atobserver", y=1.00)
        fig.savefig(f"figures/plot_{i+1}_atobserver.png")
        plt.close(fig)
        
        fig = corner.corner(np.array(coeff_array1)[:, idx], **plot_options)
        fig = corner.corner(np.array(coeff_array2)[:, idx], fig=fig, reverse=True)
        fig.suptitle("without observer (bottom), with (top)", y=1.00)
        fig.savefig(f"figures/plot_{i+1}.png")
        plt.close(fig)
        
print("done")

100%|██████████| 10000/10000 [06:51<00:00, 24.33it/s]  

done


In [40]:
def make_icrs_radial_error(r0, eps_l: float, upper: float, scaling=1):
    
    if eps_l <= 0 or 1 < eps_l:
        raise ValueError
        
    if upper <= 0 or 1 < upper:
        raise ValueError

    def icrs_radial_error(c):
        """Linearly-scaled errors in a ICRS frame.

        Transforms the samples to ICRS, assigns a distance-dependent error in the radial coordinate

        [(1 - eps_u) (d / r0) + eps_l] / (1 + d / r0)

        Parameters
        ----------
        c : SkyCoord
            in Galactocentric coordinates

        """
        cc = c.transform_to(coord.ICRS())
        
        d = cc.distance

        # reshape "c" to Nx3 array
        nd = cc.shape[0]  # the number of samples
        vals = np.abs(cc.data._values.view(dtype=np.float64).reshape(nd, -1))

        # get scaled error
        d_pos = vals
        d_pos[:, 0] = 0 * u.deg
        d_pos[:, 1] = 0 * u.deg
        d_pos[:, 2] = scaling * (upper * (d / r0) + eps_l) / (1 + d / r0) * d.to_value()
        return d_pos

    return icrs_radial_error

# /def

In [41]:
measurer2 = MeasurementErrorSampler(
    c_err=make_icrs_radial_error(r0, eps_l=0.01, upper=0.5),
    method="Gaussian",
    frame="icrs",
    representation_type="spherical"
)
measurer

pipe2 = Pipeline(
    sampler=sampler,
    measurer=measurer2,
    fitter=fitter,
)
pipe

Pipeline:
    sampler: <discO.plugin.galpy.sample.GalpyPotentialSampler object at 0x7f9984f7a130>
    measurer: <discO.core.measurement.GaussianMeasurementError object at 0x7f9988b28f40>
    fitter: <discO.plugin.galpy.fitter.GalpySCFPotentialFitter object at 0x7f9988b31220>
	frame: <Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>
	defaults: {'Nmax': 3, 'Lmax': 3, 'scale_factor': <Quantity 16. kpc>}
    residual: None
    statistic: None

In [42]:
coeff_array1 = []
coeff_array2 = []

numphis = 27  # (N=3 x (L=3)^2)
idx = [
    i for i in np.arange(numphis)
    if i not in [1, 2, 5, 10, 11, 14, 19, 20, 23]
]

plot_options=dict(
    labels=[r"$\phi_{" + str(i) + r"}$" for i in idx],
    show_titles=True,
    quantiles=[0.16, 0.5, 0.84],
    title_kwargs={"fontsize": 12},
    title_fmt='.2e',
    label_kwargs={"fontsize":12},
    hist2d_kwargs={"fontsize":8},
    quiet=True
)

# Run for 10k samples, for 10k iterations
N = int(1e4)
iterations = int(1e4)
random=0
stride = 5000

for i, result1 in enumerate(pipe2.run(N, iterations, random=random, c_err=False)):
    # the no-observer coefficients
    coeff_array1.append(result1.fit.coefficients()["Acos"].flatten())

    result2 = pipe2(result1.sample, random=random)
    coeff_array2.append(result2.fit.coefficients()["Acos"].flatten())

    if (i + 1) % stride == 0 and (i + 1) > numphis:

        # horribly inefficient saving
        np.save("figures/atcenter2", coeff_array1)
        np.save("figures/atobserver2", coeff_array2)

        # plot
        plt.close("all")
        fig = corner.corner(np.array(coeff_array1)[:, idx], **plot_options)
        fig.suptitle("At Center", y=1.00)
        fig.savefig(f"figures/plot_{i+1}_atcenter2.png")
        plt.close(fig)

        fig = corner.corner(np.array(coeff_array2)[:, idx], **plot_options)
        fig.suptitle("atobserver", y=1.00)
        fig.savefig(f"figures/plot_{i+1}_atobserver2.png")
        plt.close(fig)

        fig = corner.corner(np.array(coeff_array1)[:, idx], **plot_options)
        fig = corner.corner(np.array(coeff_array2)[:, idx], fig=fig, reverse=True)
        fig.suptitle("without observer (bottom), with (top)", y=1.00)
        fig.savefig(f"figures/plot2_{i+1}.png")
        plt.close(fig)
        
print("done")

100%|██████████| 10000/10000 [08:32<00:00, 19.50it/s]  

done
